In [ ]:
# 위 작업 일괄 처리 (파일 압축 해제)
%%bash

# 폴더 지우기
# rm -rf sample_data/dogs-vs-cats
# 폴더 복사
cp drive/MyDrive/Colab\ Notebooks/data_files/dogs-vs-cats.zip sample_data/
# 폴더 생성
mkdir smaple_data/dogs-vs-cats
# 생성된 폴더에 압축 풀기 1
unzip sample_data/dogs-vs-cats.zip -d sample_data/dogs-vs-cats
# 압축 풀기 2
mkdir sample_data/dogs-vs-cats/train
unzip sample_data/dogs-vs-cats/train.zip -d sample_data/dogs-vs-cats
# train 폴더 안에 사진 들어감
mkdir sample_data/dogs-vs-cats/test1
unzip sample_data/dogs-vs-cats/test1.zip -d sample_data/dogs-vs-cats
# test 폴더 안에 test1 폴더 안에 사진 들어감

# 폴더 이름 변경
mv sample_data/dogs-vs-cats/test1 sample_data/dogs-vs-cats/test

In [ ]:
# 데이터 디렉터리 구성
# train_dir, test_dir, validation_dir # 훈련, 테스트, 검증 세트 폴더
# train_dir -> cats, dogs             # 범주로 사용될 폴더
# test_dir -> cats, dogs
# validation_dir -> cats, dogs

import os, shutil

src_base = 'sample_data/dogs-vs-cats'
dest_base = 'sample_data/dogs-vs-cats2'

if os.path.exists(dest_base): # 이미 존재하는 폴더라면
  shutil.rmtree(dest_base)    # 폴더 제거 후
os.makedirs(dest_base)        # 새로 폴더 만들기

# 범주를 말할때 앞 숫자는 포함, 뒤 숫자는 불포함
# train 폴더의 0 ~ 1000 : trainset, 1000 ~ 1500 : validationset, 1500 ~ 2000 : testset
# > 0 ~ 999, 1500 ~ 1499, 1500 ~ 1999
# zip을 3개로 걸음. 시작 숫자, 끝 숫자, 들어갈 곳
# > 0 ~ 1000 : train, 1000 ~ 1500 : validation, 1500 ~ 2000 : test 폴더 생성
for start, stop, path in zip([0, 1000, 1500], [1000, 1500, 2000], ['train', 'validation', 'test']):
  os.mkdir(os.path.join(dest_base, path)) # train or validation or test 폴더 생성
  for sub_path in ['cat', 'dog']:
    os.mkdir(os.path.join(dest_base, path, sub_path)) # train, validation, test 하위에 cat or dog 폴더 생성
    for idx in np.arange(start, stop):
      fname = f'{sub_path}.{idx}.jpg'   # ex) cat.1.jpg 를 위치에 맞게 저장
      shutil.copy(os.path.join(src_base, 'train', fname), os.path.join(dest_base, path, sub_path, fname))

In [ ]:
# 디렉터리 구성 결과 확인
for path in ['train', 'test', 'validation']:
  for subpath in ['cat', 'dog']:
    print(path, '/', subpath, ':', len(os.listdir(f'sample_data/dogs-vs-cats2/{path}/{subpath}')))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [ ]:
# 입력 자동화 도구 만들기
# ImageDataGenerator() 이미지를 읽어서 데이터로 만드는 도구

# 특정 디렉터리의 파일을 읽어서 모델의 입력 데이터로 변환하는 도구
train_generator = tf_keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
validation_generator = tf_keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
test_generator = tf_keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [ ]:
# 입력 데이터 구성

datasets = []
for path, generator in zip(['train', 'validation', 'test'],
                           [train_generator, validation_generator, test_generator]):
  dataset = generator.flow_from_directory(directory = f'sample_data/dogs-vs-cats2/{path}',  # 디렉터리에서 알아서 이미지 가져오기
                                          target_size=(256, 256),                           # 이미지 사이즈 일괄 맞춤
                                          batch_size=32,                                    # 한 방에 처리할 이미지 수
                                          class_mode='binary')                              # 범주 몇개냐? binary : 이진분류
  datasets.append(dataset)

In [ ]:
print(type(datasets[0]))
for x_batch, y_batch in datasets[0]:
  print(x_batch.shape, y_batch.shape)
  print(x_batch[0].max())
  print(y_batch)
  break
# (32, 256, 256, 3) 32: 한 번에 뽑은 이미지 수, 256*256 가로*세로, 3 RGB 3개 채널

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(x_batch[4])
plt.axis('off')
plt.show()

In [ ]:
# 모델 구조 설계

model = tf_keras.models.Sequential([
    tf_keras.layers.Input(shape=(256, 256, 3)),
    tf_keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),  # 256 256 32
    tf_keras.layers.MaxPooling2D((2, 2)),                                                       # 128 128 32
    tf_keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),  # 128 128 64
    tf_keras.layers.MaxPooling2D((2, 2)),                                                       # 64   64 64
    tf_keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'), # 64   64 128
    tf_keras.layers.MaxPooling2D((2, 2)),                                                       # 32   32 128

    tf_keras.layers.Flatten(),

    tf_keras.layers.Dense(units=512, activation='relu'),
    tf_keras.layers.Dense(units=256, activation='relu'),
    tf_keras.layers.Dense(units=1, activation='sigmoid')  # 2진 분류면 1, sigmoid 쓰거나, 2, softmax 사용
])

model.summary() # 가중치 수 : 67,334,209